In [7]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
sys.path.append(os.path.join(os.getcwd(), 'domain'))
from tests.domain.primary.domain.create_domain import DomainTables

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float32'
dim=3
mesh_name = 'tetrahedron.msh'

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'domain', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=4, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 3
path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 0
====> Start <=====
====> Creating Mesh <=====
gmsh:physical => triangle
gmsh:physical => tetra
node_cellid
Execution time: 0.005906 seconds
cell_cellnid
Execution time: 0.010964 seconds
cell_cellfid and boundary_faces
Execution time: 0.017833 seconds
node_bfid
Execution time: 0.018100 seconds
Start creating sub domains
Parse Input 
Start Creating SubDomains 
Make n part
Start dense to csr
Nb_vertices = 6000
size = 22800
End dense to csr
End METIS_PartGraphKway
End function
Create local cells
Create Physical faces
Create local domains struct
Create local domains struct p=0
Create map_bf_recv, shared_bf_recv
Create Tables
End Create Tables
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
Create local domains struc

In [10]:
import importlib

module = importlib.import_module("helpers.TablesTestTetra3D")
importlib.reload(module)
TablesTestTetra3D = getattr(module, "TablesTestTetra3D")

module = importlib.import_module("helpers.TetraChecker3D")
importlib.reload(module)
TetraChecker3D = getattr(module, "TetraChecker3D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TablesTestTetra3D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = TetraChecker3D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.summary()
# checker.test_face_info()
# checker.test_node_info()
# checker.test_halo_info()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 4 [P] 6 [F] Total 10
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
* [F] 1/10: Cell Vertices 1 / 6000 6000 Failed
	
	Arrays are not almost equal to 4 decimals
	
	Mismatched elements: 8 / 12 (66.7%)
	Max absolute difference: 10.5
	Max relative difference: 3.
	 x: array([[1. , 2. , 0. ],
	       [0. , 2. , 1.5],
	       [0. , 1.5, 0. ],
	       [1. , 1.5, 0. ]], dtype=float32)
	 y: array([[ 1. ,  0. ,  9. ],
	       [ 0. ,  0.5,  9. ],
	       [ 0. ,  0. , 10.5],
	       [ 1. ,  0. , 10.5]], dtype=float32)
* [F] 2/10: Cell Center 1 / 6000 6000 Failed
	
	Arrays are not almost equal to 4 decimals
	
	Mismatched elements: 2 / 3 (66.7%)
	Max absolute difference: 9.375
	Max relative difference: 13.
	 x: array([0.5  , 1.75 , 0.375], dtype=float32)
	 y: array([0.5  , 0.125, 9.75 ], dtype=float32)


False

In [8]:
unified_domain.d_cells[0].shape

(6000, 5)